## Individual Project 2

In [43]:
from bs4 import BeautifulSoup
import requests
import time
import random
import re

(1) Yelp uses GET requests for its search. Using Python or Java (Selenium is OK to use but not required and may be more cumbersome here; I didn't use Selenium), write a program that searches on yelp.com for the top-40 “Donut Shop” in the San Francisco area (no need to verify that the shop is actually selling donuts, just search for “Donut Shop”, top 40 shops according to Yelp's "Recommended" sorting). Save each search result page to disk, “sf_donut_shop_search_page_[PN].htm” (replace [PN] with the page number). Please make sure to pause after loading each page. (25 points)

In [10]:
# set the header and base URL
headers = {'User-agent': 'Mozilla/5.0'}
URL_base = 'https://www.yelp.com/search?find_desc=Donut+Shop&find_loc=San+Francisco%2C+CA+94102'

In [12]:
for i in range(0,4):
    # create page variable
    page_var = '&start='+str(i*10)
    # add page variable to the base URL
    URL = URL_base+page_var

    # request the URL page content
    page = requests.get(URL, headers=headers)

    ## save the content to a new file
    # create file name
    f_name = 'sf_donut_shop_search_page_'+str(i+1)+'.htm'
    # create and open a file
    f = open(f_name,'wb+')
    # write to the file
    f.write(page.content)
    #close the file
    f.close()
    print('page',str(i+1),'complete!')
    
    # wait before accessing next page
    time.sleep(random.randint(5,10))

page 1 complete!
page 2 complete!
page 3 complete!
page 4 complete!


(2) Using Python or Java, write new code that opens the search result pages saved in (1) and parses out all shop information (search rank, name, linked URL [this store’s Yelp URL], star rating, number of reviews, store tags, “$” signs, delivery / dine-in tags, and whether you can order through Yelp). Please be sure to skip all “Sponsored Results”. (15 points)

In [154]:
# create an empty list to store the 4 beatifulsoup objects
soup_list = []
for p in range(1,5):
    # open the file
    f_name = 'sf_donut_shop_search_page_'+str(p)+'.htm'
    with open(f_name, 'r', encoding='utf-8') as t:
        soup = BeautifulSoup(t.read(), 'lxml')
    # append to the list
    soup_list.append(soup)

In [183]:
# find all 40 containers
target_div = []
for i in soup_list:
    new_div = i.find_all("div",{"class":'container__09f24__mpR8_ hoverable__09f24__wQ_on margin-t3__09f24__riq4X margin-b3__09f24__l9v5d padding-t3__09f24__TMrIW padding-r3__09f24__eaF7p padding-b3__09f24__S8R2d padding-l3__09f24__IOjKY border--top__09f24__exYYb border--right__09f24__X7Tln border--bottom__09f24___mg5X border--left__09f24__DMOkM border-color--default__09f24__NPAKY'})
    # exclude sponsored resutls
    new_target_div = new_div[2:12]
    target_div.extend(new_target_div)

In [185]:
# rank and name
for i in target_div:
    h3 = i.find("h3",{"class":'css-uvzfg9'})
    rst = h3.text
    match = re.match('(\d+)\.\s(.+)',rst)
    rank = match.group(1)
    name = match.group(2)
    print(rank,name)

1 Bob’s Donuts & Pastry Shop
2 Dough
3 Happy Donut
4 Gateway Croissant
5 Hahdough
6 Johnny Doughnuts
7 Mochill Mochi Donut
8 All Star Cafe
9 Bob’s Donuts
10 Donuts & Things
11 Grand Mission Donuts & Bakery
12 Donut Farm
13 All Stars Donuts
14 Happy Donut
15 Paradise Coffee and Donuts
16 All Star Cafe
17 Jim Georgie’s Donuts
18 Happy Donuts
19 Happy Donuts
20 Allstar Donuts & Burgers
21 Choux
22 Milkbomb Ice Cream
23 Inga Donut
24 ONE65 Patisserie & Boutique
25 Cafe Madeleine
26 Fillmore Bakeshop
27 Les Gourmands Bakery
28 The Posh Bagel
29 Jane The Bakery
30 Les Croissants Cafe
31 Hot Cookie
32 Golden Gate Bakery
33 Le Bonta Italiane
34 Craftsman and Wolves
35 Benkyodo
36 Yasukochi’s Sweet Stop
37 Lilikoi
38 Latte Express
39 Star Bagel
40 Blue Bottle Coffee


In [186]:
# url
for i in target_div:
    a = i.find("a",{"class":'css-1422juy'})
    url = a.get('href')
    print(url)

/biz/bobs-donuts-and-pastry-shop-san-francisco?osq=Donut+Shop
/biz/dough-san-francisco?osq=Donut+Shop
/biz/happy-donut-san-francisco-8?osq=Donut+Shop
/biz/gateway-croissant-san-francisco-2?osq=Donut+Shop
/biz/hahdough-san-francisco-5?osq=Donut+Shop
/biz/johnny-doughnuts-san-francisco-4?osq=Donut+Shop
/biz/mochill-mochi-donut-san-francisco?osq=Donut+Shop
/biz/all-star-cafe-san-francisco-3?osq=Donut+Shop
/biz/bobs-donuts-san-francisco-2?osq=Donut+Shop
/biz/donuts-and-things-san-francisco?osq=Donut+Shop
/biz/grand-mission-donuts-and-bakery-san-francisco?osq=Donut+Shop
/biz/donut-farm-san-francisco?osq=Donut+Shop
/biz/all-stars-donuts-san-francisco?osq=Donut+Shop
/biz/happy-donut-san-francisco-7?osq=Donut+Shop
/biz/paradise-coffee-and-donuts-san-francisco?osq=Donut+Shop
/biz/all-star-cafe-san-francisco?osq=Donut+Shop
/biz/jim-georgies-donuts-san-francisco-2?osq=Donut+Shop
/biz/happy-donuts-san-francisco-19?osq=Donut+Shop
/biz/happy-donuts-san-francisco-4?osq=Donut+Shop
/biz/allstar-donuts-

In [205]:
# star
for i in target_div:
    span = i.find("span",{"class":'display--inline__09f24__c6N_k border-color--default__09f24__NPAKY'})
    div = span.find('div')
    label = div.get('aria-label')
    match = re.match('(\d\.*\d*)(.*)',label)
    star = match.group(1)
    print(star)

4.5
4.5
4
3.5
4.5
3.5
4.5
4
4.5
4.5
4
4
3.5
4
3.5
3.5
3.5
3.5
3.5
3.5
4
4.5
3
4.5
4
4
4.5
3.5
4.5
3.5
4
4.5
4.5
4
4.5
4.5
4.5
3.5
3.5
4


In [188]:
# number of reviews
for i in target_div:
    span = i.find("span",{"class":'reviewCount__09f24__tnBk4 css-1e4fdj9'})
    review_num = span.text
    print(review_num)

3328
73
42
125
72
220
819
208
117
119
104
319
129
185
6
134
26
218
73
28
302
701
37
314
494
550
386
388
478
46
1540
3395
12
1641
1190
729
127
119
121
2244


In [189]:
# store tags
tag_list = []
for i in target_div:
    tag_button = i.find_all("p",{"class":'css-1p8aobs'})
    new = []
    for t in tag_button:
        tags = t.text
        new.append(tags)
    tag_list.append(new)

In [192]:
tag_list

[['Bakeries', 'Donuts'],
 ['Donuts', 'Pizza'],
 ['Donuts'],
 ['Coffee & Tea', 'Donuts', 'Bagels'],
 ['Donuts', 'Patisserie/Cake Shop', 'Pretzels'],
 ['Donuts', 'Coffee & Tea', 'Food Trucks'],
 ['Donuts'],
 ['Donuts', 'Coffee & Tea', 'Breakfast & Brunch'],
 ['Donuts', 'Bakeries'],
 ['Donuts', 'Coffee & Tea'],
 ['Donuts', 'Bakeries'],
 ['Donuts', 'Vegan'],
 ['Donuts', 'Coffee & Tea', 'Fast Food'],
 ['Donuts', 'Coffee & Tea', 'Sandwiches'],
 ['Donuts'],
 ['Coffee & Tea', 'Cafes'],
 ['Donuts'],
 ['Donuts', 'Vietnamese', 'Thai'],
 ['Donuts', 'Bagels', 'Breakfast & Brunch'],
 ['Burgers', 'Donuts'],
 ['Bakeries', 'Desserts', 'French'],
 ['Ice Cream & Frozen Yogurt', 'Donuts'],
 ['Donuts', 'Chinese'],
 ['Patisserie/Cake Shop', 'Bakeries', 'Coffee & Tea'],
 ['Bakeries', 'Coffee & Tea', 'Sandwiches'],
 ['Desserts', 'Bakeries'],
 ['Bakeries', 'Coffee & Tea', 'Patisserie/Cake Shop'],
 ['Bakeries', 'Bagels', 'Coffee & Tea'],
 ['Bakeries', 'Coffee & Tea', 'Breakfast & Brunch'],
 ['Coffee & Tea', 'Do

In [194]:
# “$” signs
for i in target_div:
    span = i.find("span",{"class":"priceRange__09f24__mmOuH css-18qxe2r"})
    try:
        price = span.text
        print(price)
    except:
        print('Null')

$
Null
$
$
$$
$$
Null
$
$
$
$
$
$
$
$
$
$
$
$
$
$
$$
$
$$
$
$$
$$
$$
$
$
$$
$
$
$$
$$
$$
Null
$
$
$$


In [195]:
# delivery / dine-in tags
for i in target_div:
    div = i.find_all("div",{"class":"tag__09f24__mjBHZ css-1v994a0 margin-t0-5__09f24__gboxT border-color--default__09f24__NPAKY"})
    index_list = []
    value_list = []
    for t in div:
        try:
            index = t.find("span",{"class":"raw__09f24__T4Ezm"}).text
            index_list.append(index)
            true = t.find("span",{"class":"icon--16-checkmark-v2 css-1z0vsn7"})
            if true == None:
                value = 0
            else:
                value = 1
            value_list.append(value)
        except:
            None
    print(index_list,value_list)

[] []
['Delivery', 'Takeout'] [1, 1]
[] []
['Outdoor dining', 'Delivery', 'Takeout'] [0, 1, 1]
['Delivery'] [1]
['Outdoor dining', 'Delivery', 'Takeout'] [1, 1, 1]
['Takeout'] [1]
['Delivery', 'Takeout'] [1, 1]
['Takeout'] [1]
['Outdoor dining', 'Delivery', 'Takeout'] [0, 0, 1]
[] []
['Takeout'] [1]
[] []
[] []
[] []
['Delivery', 'Takeout'] [1, 1]
[] []
[] []
[] []
[] []
['Outdoor dining', 'Delivery', 'Takeout'] [0, 1, 1]
['Delivery', 'Takeout', 'Curbside Pickup'] [1, 1, 0]
[] []
['Delivery', 'Takeout', 'Curbside Pickup'] [1, 1, 0]
['Outdoor dining', 'Delivery', 'Takeout'] [1, 1, 1]
['Delivery', 'Takeout'] [0, 1]
[] []
['Outdoor dining', 'Delivery', 'Takeout'] [0, 0, 1]
['Delivery', 'Takeout'] [1, 1]
[] []
['Delivery', 'Takeout', 'Curbside Pickup'] [1, 1, 1]
['Takeout'] [1]
[] []
['Outdoor dining', 'Delivery', 'Takeout'] [1, 1, 1]
[] []
[] []
['Proof of vaccination required', 'Delivery', 'Takeout', 'Curbside Pickup'] [0, 1, 1, 1]
[] []
[] []
['Delivery', 'Takeout'] [1, 0]


In [196]:
# whether you can order through Yelp
for i in target_div:
    span = i.find("span",{"class":"css-1enow5j"})
    if span == None:
        order = 0
    else:
        order =1
    print(order)

0
1
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
1
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0


(3) Adjust your code in (2) to create a MongoDB collection called “sf_donut_shops” that stores all the extracted shop information, one document for each shop. (10 points)

In [197]:
import pymongo

In [200]:
# create a MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase"]

In [201]:
# create a collection
sf_donut_shops = mydb["sf_donut_shops"]

In [215]:
# create a list of documents
doc_list = []
for i in target_div:
    new_dic = {}
    
    # insert rank and name
    h3 = i.find("h3",{"class":'css-uvzfg9'})
    rst = h3.text
    match = re.match('(\d+)\.\s(.+)',rst)
    rank = match.group(1)
    name = match.group(2)
    new_dic['rank'] = rank
    new_dic['name'] = name
    
    # insert url
    a = i.find("a",{"class":'css-1422juy'})
    url = a.get('href')
    new_dic['url'] = url
    
    # insert number of stars
    span = i.find("span",{"class":'display--inline__09f24__c6N_k border-color--default__09f24__NPAKY'})
    div = span.find('div')
    label = div.get('aria-label')
    match = re.match('(\d\.*\d*)(.*)',label)
    star = match.group(1)
    new_dic['stars'] =  star
    
    # insert number of reviews
    span2 = i.find("span",{"class":'reviewCount__09f24__tnBk4 css-1e4fdj9'})
    review_num = span2.text
    new_dic['reviews'] = review_num
    
    # insert store tags
    tag_button = i.find_all("p",{"class":'css-1p8aobs'})
    new = []
    for t in tag_button:
        tags = t.text
        new.append(tags)
    sep = '|'
    tags = sep.join(new)
    new_dic['tags'] = tags
    
    # insert $ signs
    span3 = i.find("span",{"class":"priceRange__09f24__mmOuH css-18qxe2r"})
    try:
        price = span3.text
    except:
        price = None
    new_dic['price'] = price
    
    # insert Outdoor dining / delivery / takeout tags
    new_dic['dining'] = None
    new_dic['delivery'] = None
    new_dic['takeout'] = None
    div2 = i.find_all("div",{"class":"tag__09f24__mjBHZ css-1v994a0 margin-t0-5__09f24__gboxT border-color--default__09f24__NPAKY"})
    index_list = []
    value_list = []
    for t in div2:
        try:
            index = t.find("span",{"class":"raw__09f24__T4Ezm"}).text
            index_list.append(index)
            true = t.find("span",{"class":"icon--16-checkmark-v2 css-1z0vsn7"})
            if true == None:
                value = 0
            else:
                value = 1
            value_list.append(value)
        except:
            None
    if len(index_list) != 0:
        for l in range(0,len(index_list)):
            if index_list[l] == 'Outdoor dining':
                new_dic['dining'] = value_list[l]
            if index_list[l] == 'Delivery':
                new_dic['delivery'] = value_list[l]
            if index_list[l] == 'Takeout':
                new_dic['takeout'] = value_list[l]
    
    # insert whether you can order through Yelp
    span4 = i.find("span",{"class":"css-1enow5j"})
    if span4 == None:
        new_dic['yelp'] = 0
    else:
        new_dic['yelp'] = 1
    
    # append new document to the list
    doc_list.append(new_dic)

In [219]:
# insert into the collection
rst = sf_donut_shops.insert_many(doc_list)

In [229]:
# look at the whole collection
for x in sf_donut_shops.find():
    print(x)

{'_id': ObjectId('62204089fb45f97290ef646f'), 'rank': '1', 'name': 'Bob’s Donuts & Pastry Shop', 'url': '/biz/bobs-donuts-and-pastry-shop-san-francisco?osq=Donut+Shop', 'stars': '4.5', 'reviews': '3328', 'tags': 'Bakeries|Donuts', 'price': '$', 'dining': None, 'delivery': None, 'takeout': None, 'yelp': 0}
{'_id': ObjectId('62204089fb45f97290ef6470'), 'rank': '2', 'name': 'Dough', 'url': '/biz/dough-san-francisco?osq=Donut+Shop', 'stars': '4.5', 'reviews': '73', 'tags': 'Donuts|Pizza', 'price': None, 'dining': None, 'delivery': 1, 'takeout': 1, 'yelp': 1}
{'_id': ObjectId('62204089fb45f97290ef6471'), 'rank': '3', 'name': 'Happy Donut', 'url': '/biz/happy-donut-san-francisco-8?osq=Donut+Shop', 'stars': '4', 'reviews': '42', 'tags': 'Donuts', 'price': '$', 'dining': None, 'delivery': None, 'takeout': None, 'yelp': 0}
{'_id': ObjectId('62204089fb45f97290ef6472'), 'rank': '4', 'name': 'Gateway Croissant', 'url': '/biz/gateway-croissant-san-francisco-2?osq=Donut+Shop', 'stars': '3.5', 'revie

(4) Using Python or Java, write a _new_ piece of code that reads all URLs stored in “sf_donut_shops” and download each shop page. Store the page to disk, “sf_donut_shop_[SR].htm” (replace [SR] with the search rank). (15 points)

In [251]:
# select the url field
url_list = []
for x in sf_donut_shops.find({},{"_id": 0,"url": 1}):
    url_list.append('https://www.yelp.com/' + x['url'])

In [259]:
for i in range(0,len(url_list)):
    # request the URL page content
    page = requests.get(url_list[i], headers=headers)

    ## save the content to a new file
    # create file name
    f_name = 'sf_donut_shop_'+str(i+1)+'.htm'
    # create and open a file
    f = open(f_name,'wb+')
    # write to the file
    f.write(page.content)
    #close the file
    f.close()
    print('shop',str(i+1),'complete!')
    
    # wait before accessing next page
    time.sleep(30)

shop 1 complete!
shop 2 complete!
shop 3 complete!
shop 4 complete!
shop 5 complete!
shop 6 complete!
shop 7 complete!
shop 8 complete!
shop 9 complete!
shop 10 complete!
shop 11 complete!
shop 12 complete!
shop 13 complete!
shop 14 complete!
shop 15 complete!
shop 16 complete!
shop 17 complete!
shop 18 complete!
shop 19 complete!
shop 20 complete!
shop 21 complete!
shop 22 complete!
shop 23 complete!
shop 24 complete!
shop 25 complete!
shop 26 complete!
shop 27 complete!
shop 28 complete!
shop 29 complete!
shop 30 complete!
shop 31 complete!
shop 32 complete!
shop 33 complete!
shop 34 complete!
shop 35 complete!
shop 36 complete!
shop 37 complete!
shop 38 complete!
shop 39 complete!
shop 40 complete!


(5) Using Python or Java, write new code that reads the 40 shop pages saved in (4) and parses each shop’s address, phone number, and website. (15 points)

In [260]:
# create an empty list to store the 40 beatifulsoup objects
shop_list = []
for p in range(1,41):
    # open the file
    f_name = 'sf_donut_shop_'+str(p)+'.htm'
    with open(f_name, 'r', encoding='utf-8') as t:
        shop = BeautifulSoup(t.read(), 'lxml')
    # append to the list
    shop_list.append(shop)

In [263]:
try1 = shop_list[0]

In [270]:
# parse address
for i in shop_list:
    p = i.find("p",{"class":"css-qyp8bo"})
    address = p.text
    print(address)

1621 Polk St San Francisco, CA 94109
1455 Market St San Francisco, CA 94103
299 Ellis St San Francisco, CA 94102
390 Golden Gate Ave San Francisco, CA 94102
509 Laguna St San Francisco, CA 94102
392 Fulton St San Francisco, CA 94102
1737 Post St Ste 395 San Francisco, CA 94115
98 9th St San Francisco, CA 94103
601 Baker St San Francisco, CA 94117
1549 Polk St San Francisco, CA 94109
2195 Mission St San Francisco, CA 94110
1 Ferry Bldg Ste 14 San Francisco, CA 94111
399 5th St San Francisco, CA 94107
145 Columbus Ave San Francisco, CA 94133
453 Ofarrell St San Francisco, CA 94102
1500 Market St San Francisco, CA 94102
2799 16th St San Francisco, CA 94103
100 Bush St San Francisco, CA 94111
2101 Van Ness Ave San Francisco, CA 94109
290 Golden Gate Ave San Francisco, CA 94102
248 Fillmore St San Francisco, CA 94117
1717 17th St Ste 105 San Francisco, CA 94103
1026 Hyde St San Francisco, CA 94109
165 O'Farrell St San Francisco, CA 94102
149 New Montgomery St San Francisco, CA 94105
1890 Fi

In [410]:
# parse phone number
phone_list = []
for i in shop_list:
    p = i.find_all("section",{"class":"margin-b3__09f24__l9v5d border-color--default__09f24__NPAKY"})
    target = p[1]
    # find whether phone number exist
    exist_tag = target.find_all("p",{"class":"css-na3oda"})
    if len(exist_tag) == 0:
        phone = None
        phone_list.append(phone)
        continue
    else:
        for t in exist_tag:
            tag = t.text
            if tag == 'Phone number':
                # phone number tag exist, let's find the phone number
                new_p = target.find_all("p",{"class":"css-1p9ibgf"})
                for x in new_p:
                    try:
                        rst = x.text
                        match = re.match('(\(\d+\)\s.+)',rst)
                        phone = match.group(1)
                    except:
                        continue
            else:
                phone = None
                continue
        phone_list.append(phone)
phone_list

['(415) 776-3141',
 '(415) 864-0888',
 '(415) 440-1010',
 '(415) 749-0202',
 '(415) 676-0576',
 '(415) 400-5078',
 '(650) 619-2462',
 '(415) 863-4949',
 '(415) 776-3141',
 '(415) 885-1942',
 None,
 None,
 '(415) 882-0889',
 '(415) 956-9189',
 '(415) 673-7003',
 '(415) 252-9888',
 '(415) 861-7112',
 '(415) 398-6769',
 '(415) 800-7399',
 '(415) 921-5541',
 '(628) 218-6820',
 '(415) 903-4281',
 '(415) 771-9611',
 '(415) 814-8888',
 '(415) 227-4077',
 '(415) 923-0711',
 '(415) 872-9461',
 '(415) 951-0133',
 '(415) 658-7971',
 '(415) 292-6688',
 '(415) 621-2350',
 '(415) 781-2627',
 None,
 '(415) 913-7713',
 '(415) 922-1244',
 '(415) 931-8165',
 '(415) 529-1496',
 '(415) 348-8898',
 '(415) 567-9955',
 '(510) 653-3394']

In [295]:
# parse website
for i in shop_list:
    a = i.find("a",{"class":"css-1um3nx","target":"_blank"})
    try:
        url = a.text
        print(url)
    except:
        print(None)

http://www.bobsdonutssf.com
https://www.sfdough.com
None
http://gatewaycroissantsanfran…
http://hahdough.com
http://johnnydoughnuts.com
https://www.mochillsf.com
http://allstarcafe.top-cafes.c…
https://www.bobsdonutssf.com
None
None
https://www.vegandonut.farm
None
None
None
None
None
None
http://happydonutssanfrancisco…
None
http://www.chouxsf.com
https://www.milkbombicecream.c…
None
https://one65sf.com
http://www.cafemadeleinesf.com
http://fillmorebakeshop.com/
None
http://theposhbagelsanfrancisc…
https://www.itsjane.com
None
http://www.hotcookie.com
None
None
http://craftsman-wolves.com
http://www.benkyodocompany.com…
https://www.sweetstopsf.com
https://lilikoiboba.com
None
None
https://bluebottlecoffee.com


(6) Sign up for a free account with https://positionstack.com/ Adjust your code in (5) to query each shop address’ geolocation (long, lat). Update each shop document on the MongoDB collection “sf_donut_shops” to contain the shop’s address, phone number, website, and geolocation. Lastly, place an index on the shop’s search rank. (20 points)

In [302]:
import json

In [300]:
# set up API access indormation
base_url = 'http://api.positionstack.com/v1/forward?access_key=6fd8fed2f3d30760d95c7bbc83864ac3&output=json'

In [297]:
# create a list of all addresses
address_list = []
for i in shop_list:
    p = i.find("p",{"class":"css-qyp8bo"})
    address = p.text
    address_list.append(address)

In [317]:
# query the geolocation information from the API
json_dict = []
for a in address_list:
    url = base_url + '&query=' + a
    page= requests.get(url,headers=headers)
    doc= BeautifulSoup(page.content, 'html.parser')
    json_dict.append(json.loads(str(doc))['data'][0])

In [331]:
# creat a list of geolocation infromation
geo_list = []
for i in json_dict:
    new_dic = {}
    new_dic['latitude'] = i['latitude']
    new_dic['longitude'] = i['longitude']
    geo_list.append(new_dic)

In [412]:
# creat a list of phone number
phone_list = []
for i in shop_list:
    p = i.find_all("section",{"class":"margin-b3__09f24__l9v5d border-color--default__09f24__NPAKY"})
    target = p[1]
    # find whether phone number exist
    exist_tag = target.find_all("p",{"class":"css-na3oda"})
    if len(exist_tag) == 0:
        phone = None
        phone_list.append(phone)
        continue
    else:
        for t in exist_tag:
            tag = t.text
            if tag == 'Phone number':
                # phone number tag exist, let's find the phone number
                new_p = target.find_all("p",{"class":"css-1p9ibgf"})
                for x in new_p:
                    try:
                        rst = x.text
                        match = re.match('(\(\d+\)\s.+)',rst)
                        phone = match.group(1)
                    except:
                        continue
            else:
                phone = None
                continue
        phone_list.append(phone)

In [335]:
# create a list of website
web_list = []
for i in shop_list:
    a = i.find("a",{"class":"css-1um3nx","target":"_blank"})
    try:
        url = a.text
        new = url
    except:
        new = None
    web_list.append(new)

In [413]:
# Update address, phone number, website, and geolocation to "sf_donut_shops" collection
n=0
for c in sf_donut_shops.find():
    sf_donut_shops.update_one({'_id':c['_id']},{'$set' : {'address':address_list[n]}})
    sf_donut_shops.update_one({'_id':c['_id']},{'$set' : {'phone':phone_list[n]}})
    sf_donut_shops.update_one({'_id':c['_id']},{'$set' : {'website':web_list[n]}})
    sf_donut_shops.update_one({'_id':c['_id']},{'$set' : {'geolocation':geo_list[n]}})
    n = n + 1

In [414]:
# have a look at the updated "sf_donut_shops" collection
for i in sf_donut_shops.find():
    print(i)

{'_id': ObjectId('62204089fb45f97290ef646f'), 'rank': '1', 'name': 'Bob’s Donuts & Pastry Shop', 'url': '/biz/bobs-donuts-and-pastry-shop-san-francisco?osq=Donut+Shop', 'stars': '4.5', 'reviews': '3328', 'tags': 'Bakeries|Donuts', 'price': '$', 'dining': None, 'delivery': None, 'takeout': None, 'yelp': 0, 'address': '1621 Polk St San Francisco, CA 94109', 'phone': '(415) 776-3141', 'website': 'http://www.bobsdonutssf.com', 'geolocation': {'latitude': 37.791852, 'longitude': -122.421276}}
{'_id': ObjectId('62204089fb45f97290ef6470'), 'rank': '2', 'name': 'Dough', 'url': '/biz/dough-san-francisco?osq=Donut+Shop', 'stars': '4.5', 'reviews': '73', 'tags': 'Donuts|Pizza', 'price': None, 'dining': None, 'delivery': 1, 'takeout': 1, 'yelp': 1, 'address': '1455 Market St San Francisco, CA 94103', 'phone': '(415) 864-0888', 'website': 'https://www.sfdough.com', 'geolocation': {'latitude': 37.775238, 'longitude': -122.417523}}
{'_id': ObjectId('62204089fb45f97290ef6471'), 'rank': '3', 'name': 'H

In [416]:
# place an index on the shop’s search rank
sf_donut_shops.create_index('rank')
# Get indexes
sf_donut_shops.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'rank_1': {'v': 2, 'key': [('rank', 1)]}}